In [5]:
!pip install category_encoders

In [6]:
import pandas as pd
import sys
from datetime import datetime
import re
import category_encoders as ce
if sys.platform == 'win32':
    path = ''
else:
  from google.colab import drive
  drive.mount('/content/drive')
  path = "drive/MyDrive/Colab Notebooks/Diplom/project/"

 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def age(df):
  date_dict = {'январь':'january','февраль':'february','март':'march',	
          	'апрель':'april','май': 'may','июнь':	'june',
          	'июль':'july','август':'august','сентябрь':'september',
	          'октябрь':'october','ноябрь':'november','декабрь':'december'} 
  df.age =  df.age.apply(lambda x: date_dict[x.split()[0]] + '/' + \
                         x.split()[1] if x==x else x)
  df.age = df.age.apply(lambda x:int(round((datetime.today() - \
          datetime.strptime(x,'%B/%Y')).days/30,0)) if x == x else x) 
  
def to_number(df,cols):
  for c in cols:
      df[c[0]] = df[c[0]].apply(lambda x: x.split()[0] if x==x else c[1] )
      df[c[0]] = pd.to_numeric(df[c[0]])

def to_bin(df,cols):
  for c in cols:
    df[c] = df[c].apply(lambda x: 1 if x == x else 0)

def m2_regex(x):
    if x == x:
      if len(x) > 0:
        return int(x[0][0])
      else:
        return 0
    else:
      return 0   

def smart_fillna(df,inplace_col,example_col):
  for c in list(df[example_col].unique()):
    try:
      df[inplace_col].mask((df[inplace_col].isna() & (df[example_col] == c)),
                        df[inplace_col][df[example_col] == c].mode()[0],
                        inplace=True)
    except:
      pass  

def extract_dimensions(df):
  
  df['height'] = df.dimensions.str.findall(r'\d+').apply(
                              lambda x: int(x[0]) if x == x else x)
  df['width'] = df.dimensions.str.findall(r'\d+').apply(
                              lambda x: int(x[1]) if x == x else x)
  try:
    df['deep'] = df.dimensions.str.findall(r'\d+').apply(
                              lambda x: int(x[2]) if x == x else x)
  except:
    pass  
  df.drop('dimensions',axis = 1, inplace = True )
  
def full_diameter(vent):
  out = 0
  if vent == vent:
    for x in vent:
      out += int(re.split(r'\D',x)[0]) * int(re.split(r'\D',x)[1])
  return out

In [8]:
proc = pd.read_csv(path + 'data/raw_data/proc.csv')
proc = proc[['id','name', 'price', 'link','overclock', 'Серия', 'Кодовое название', 'Разъем (Socket)',
       'Техпроцесс', 'Кол-во ядер', 'Кол-во потоков', 'Тактовая частота',
       'Частота TurboBoost / TurboCore', '3-го уровня L3', 'Модель IGP',
       'Тепловыделение (TDP)', 'Макс. рабочая температура', 'Макс. объем',
       'Макс. частота DDR3', 'Макс. частота DDR4', 'Число каналов',
       'Дата добавления на E-Katalog']]
proc.columns = ['id','name', 'price', 'link','overclock', 'series', 'code_name',
                      'socket','techprocess', 'cores', 'threads', 'freq',
                      'turbo_freq', 'cashe_l3', 'igp','tdp','max_temp',
                      'max_ram','ddr3', 'ddr4', 'channels','age']
proc = proc[~proc.ddr3.isna() | ~proc.ddr4.isna()]
proc.dropna(subset=['techprocess',
                    'threads','freq'],inplace=True)
proc.turbo_freq.fillna(proc.freq,inplace=True)
proc.overclock = proc.overclock.fillna(0).apply(int)
age(proc)
proc.igp.fillna('отсутствует',inplace = True)

"""I've decided to leave NaN's in some cases, 
to choose what to do with them during model training."""

to_number(proc,[['techprocess'], ['cores',None,], ['threads'], ['freq'],
                ['turbo_freq'],['cashe_l3',0], ['tdp',None],['max_temp',None],
                ['max_ram',None], ['ddr3',0],['ddr4',0], ['channels',None]
                ]) 

proc['ram_max_freq'] = proc.ddr3.combine(proc.ddr4,max)
proc.ddr3 = proc.ddr3.apply(lambda x: 1 if x!=0 else 0)
proc.ddr4 = proc.ddr4.apply(lambda x: 1 if x!=0 else 0)
proc.to_csv(path + 'data/datasets/proc.csv',index = False)

display(proc.head(3))
proc.info()


,id,name,price,link,overclock,series,code_name,socket,techprocess,cores,threads,freq,turbo_freq,cashe_l3,igp,tdp,max_temp,max_ram,ddr3,ddr4,channels,age,ram_max_freq
0,INTEL-E3-1240-V6-BOX,Intel Xeon E3 v6 E3-1240 v6 BOX (BX80677E31240V6),23178,https://www.e-katalog.ru/INTEL-E3-1240-V6-BOX.htm,0,Xeon,Kaby Lake,Intel LGA 1151,14,4,8,3.7,4.1,8.0,отсутствует,72,NaN,64.0,1,1,2.0,56,2400
1,INTEL-3206R,Intel Xeon Bronze 3206R,27216,https://www.e-katalog.ru/INTEL-3206R.htm,0,Xeon,Cascade Lake,Intel LGA 3647,14,8,8,1.9,1.9,11.0,отсутствует,85,78.0,1000.0,0,1,6.0,12,2133
2,INTEL-I5-4690S,Intel Core i5 Haswell i5-4690S,11499,https://www.e-katalog.ru/INTEL-I5-4690S.htm,0,Core i5,Haswell,Intel LGA 1150,22,4,4,3.2,3.9,6.0,HD Graphics 4600,65,71.0,32.0,1,0,2.0,94,1600


<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 0 to 637
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            569 non-null    object 
 1   name          569 non-null    object 
 2   price         569 non-null    int64  
 3   link          569 non-null    object 
 4   overclock     569 non-null    int64  
 5   series        568 non-null    object 
 6   code_name     569 non-null    object 
 7   socket        569 non-null    object 
 8   techprocess   569 non-null    int64  
 9   cores         569 non-null    int64  
 10  threads       569 non-null    int64  
 11  freq          569 non-null    float64
 12  turbo_freq    569 non-null    float64
 13  cashe_l3      569 non-null    float64
 14  igp           569 non-null    object 
 15  tdp           569 non-null    int64  
 16  max_temp      421 non-null    float64
 17  max_ram       458 non-null    float64
 18  ddr3          569 non-null    

In [9]:
mb = pd.read_csv(path + 'data/raw_data/mb.csv')
try:
  mb = mb[mb.DDR2.isna() &
          mb['IDE разъем'].isna() &
          mb['Модель встроенного процессора'].isna()]
except:
  pass

mb.dropna(subset = ['Socket','Чипсет*'],inplace=True)  
mb = mb[['id', 'name', 'price', 'link', 'По направлению',
'Дата добавления на E-Katalog', 'Socket','Форм-фактор','Размеры (ВхШ)',
'Чипсет*','DDR3','DDR4','DDR5','Форм-фактор слота для памяти',
'Режим работы','Максимальная тактовая частота', 'Максимальный объем памяти',
'Выход DVI', 'Звук (каналов)','SATA2 (3 Гбит/с)',
'SATA3 (6 Гбит/с)','M.2 разъем','Интерфейс M.2', 'LAN (RJ-45)','Кол-во LAN-портов',
'Поддержка PCI Express','Слотов PCI-E 1x','Слотов PCI-E 4x','Слотов PCI-E 8x',
'Слотов PCI-E 16x','PCI-слотов','USB 2.0','USB 3.2 gen1','USB 3.2 gen2',
'USB C 3.2 gen1','USB C 3.2 gen2','USB C 3.2 gen2x2','PS/2', 'Фазы питания',
'Основной разъем питания','Питание процессора', 'Разъемов питания кулеров',
'Версия DisplayPort','Интегрированный RAID контроллер','Интерфейс Thunderbolt',
'Разъем Thunderbolt AIC','Синхронизация подсветки','ARGB LED strip','RGB LED strip',
'CPU Fan 4-pin', 'CPU/Water Pump Fan 4-pin','Chassis/Water Pump Fan 4-pin',
'Усилитель', 'Wi-Fi', 'Модель встроенной видеокарты',"Кол-во socket'ов",
'U.2 разъем','Bluetooth','USB 2.0*','USB 3.2 gen1*','USB C 3.2 gen2*',
 'USB 3.2 gen2*','USB C 3.2 gen2x2*','USB C 3.2 gen1*']]

mb.columns =['id', 'name', 'price', 'link', 'direction','age', 'socket',
'form','dimensions', 'chipset','ddr3','ddr4','ddr5','dimm',
'ram_channels','max_ram_freq', 'max_ram_vol','dvi','audio','sata2',
'sata3','m2','m2_interface', 'lan_speed','lan_ports','pci_e_support','pci_e_1x',
'pci_e_4x','pci_e_8x', 'pci_e_16x','pci','usb2','usb_3.2_g1','usb_3.2_g2','usb-c_3.2_g1',
'usb-c_3.2_g2', 'usb-c_3.2_g2x2','ps/2', 'power_phases','power_connector',
'power_processor', 'power_coolers','display_port','raid','thunder_interface',
'thunder_connector','light_sync','argb_led','rgb_led','cpu_fan', 'cpu_water_fan',
'chassis_water_fan','amplifier', 'wifi', 'video_int',"2*socket", 'u_2','bluetooth',
 'USB 2.0*', 'USB 3.2 gen1*', 'USB C 3.2 gen2*', 'USB 3.2 gen2*', 'USB C 3.2 gen2x2*',
 'USB C 3.2 gen1*']

age(mb)

for c in ['socket','power_processor','direction']:
  mb[c] = mb[c].apply(lambda x:x.split(' /')[0] if x == x else x)

smart_fillna(mb,'dimensions','form')

for c in ['ddr3','ddr4','ddr5','thunder_connector','thunder_interface','raid']:
  mb[c].fillna('',inplace=True)

mb['ram_slots'] = mb.ddr3 + mb.ddr4 + mb.ddr5
mb.ram_slots = mb.ram_slots.apply(lambda x: int(x.split()[0]))

for d in ['ddr3','ddr4','ddr5']: 
  mb[d] = mb[d].apply(lambda x: 0 if x == '' else 1)

mb.dimm = mb.dimm.apply(lambda x: 1 if x == 'DIMM' else 0)
mb.ram_channels.replace('2-х/4-х канальный','4',inplace=True)
mb.ram_channels = mb.ram_channels.apply(lambda x: int(x[0]) if x == x else 2 )

to_bin(mb,['dvi','light_sync','amplifier','wifi','video_int','2*socket','bluetooth'])


to_number(mb,[['audio',0],['sata2',0],['sata3',0],['m2',0],
              ['max_ram_freq',None],['max_ram_vol',None],['lan_speed',0],
              ['lan_ports',0],['pci_e_1x',0], ['pci_e_4x',0], ['pci_e_8x',0],
              ['pci_e_16x',0],['pci',0], ['usb2',0], ['usb_3.2_g1',0], 
              ['usb_3.2_g2',0],['usb-c_3.2_g1',0],['usb-c_3.2_g2',0],
              ['usb-c_3.2_g2x2',0],['ps/2',0],['power_phases',None],
              ['power_coolers',0],['rgb_led',0],['argb_led',0],['cpu_fan',0],
              ['cpu_water_fan',0],['chassis_water_fan',0], ['u_2',0],
              ['USB 2.0*',0], ['USB 3.2 gen1*',0], ['USB C 3.2 gen2*',0], 
              ['USB 3.2 gen2*',0],['USB C 3.2 gen2x2*',0],['USB C 3.2 gen1*',0]
              ])

mb['m2_pci_e_2x'] = mb.m2_interface.str.findall(r'\dxPCI-E 2x')
mb.m2_pci_e_2x = mb.m2_pci_e_2x.apply(lambda x:m2_regex(x))
mb['m2_pci_e_4x'] = mb.m2_interface.str.findall(r'\dxPCI-E 4x')
mb.m2_pci_e_4x = mb.m2_pci_e_4x.apply(lambda x:m2_regex(x))
mb['m2_sata_2x'] = mb.m2_interface.str.findall(r'\dxSATA/PCI-E 2x')
mb.m2_sata_2x = mb.m2_sata_2x.apply(lambda x:m2_regex(x))
mb['m2_sata_4x'] = mb.m2_interface.str.findall(r'\dxSATA/PCI-E 4x')
mb.m2_sata_4x = mb.m2_sata_4x.apply(lambda x:m2_regex(x))

mb.pci_e_support = mb.pci_e_support.fillna(0).apply(int)

mb.display_port = mb.display_port.apply(lambda x: 1 if x==x else 0)

mb['raid_0_1'] = mb.raid.apply(lambda x: 1 if x == x else 0)
mb['raid_5'] = mb.raid.apply(lambda x: 1 if '5' in x else 0)
mb['raid_10'] = mb.raid.apply(lambda x: 1 if '10' in x else 0)

mb['thunder_v3'] = (mb.thunder_interface + mb.thunder_connector).apply\
                    (lambda x: int(x.split(' шт')[-2][-1]) if 'v3' in x else 0)
mb['thunder_v4'] = (mb.thunder_interface + mb.thunder_connector).apply\
                    (lambda x: int(x.split(' шт')[-2][-1]) if 'v4' in x else 0)

mb.drop(['thunder_interface','thunder_connector',
         'm2_interface'],axis=1,inplace=True)                                        
extract_dimensions(mb)
mb.to_csv(path + 'data/datasets/mb.csv',index = False)

display(mb.sample(5))
mb.info()

,id,name,price,link,direction,age,socket,form,chipset,ddr3,ddr4,ddr5,dimm,ram_channels,max_ram_freq,max_ram_vol,dvi,audio,sata2,sata3,m2,lan_speed,lan_ports,pci_e_support,pci_e_1x,pci_e_4x,pci_e_8x,pci_e_16x,pci,usb2,usb_3.2_g1,usb_3.2_g2,usb-c_3.2_g1,usb-c_3.2_g2,usb-c_3.2_g2x2,ps/2,power_phases,power_connector,power_processor,power_coolers,display_port,raid,light_sync,argb_led,rgb_led,cpu_fan,cpu_water_fan,chassis_water_fan,amplifier,wifi,video_int,2*socket,u_2,bluetooth,USB 2.0*,USB 3.2 gen1*,USB C 3.2 gen2*,USB 3.2 gen2*,USB C 3.2 gen2x2*,USB C 3.2 gen1*,ram_slots,m2_pci_e_2x,m2_pci_e_4x,m2_sata_2x,m2_sata_4x,raid_0_1,raid_5,raid_10,thunder_v3,thunder_v4,height,width
192,ASUS-PRIME-Z690-P-WIFI,Asus PRIME Z690-P WIFI,23101,https://www.e-katalog.ru/ASUS-PRIME-Z690-P-WIF...,игровая для разгона (overclocking),3,Intel LGA 1700,ATX,Intel Z690,0,0,1,1,2,6000.0,128.0,0,7.1,0,4,3,2.5,1,5,1,0,0,4,0,2,2,1,1,0,1,1,15.0,24-контактный,8+4-контактное,6,1,,0,3,2,1,2,3,0,1,0,0,0,1,2,2,0,0,0,0,4,0,2,0,1,1,0,0,0,0,305,234
26,ASROCK-B560M-PRO4,ASRock B560M Pro4,8671,https://www.e-katalog.ru/ASROCK-B560M-PRO4.htm,геймерская,13,Intel LGA 1200,micro-ATX,Intel B560,0,1,0,1,2,4800.0,128.0,0,7.1,0,6,2,1.0,1,4,1,0,0,2,0,2,4,0,1,0,0,1,8.0,24-контактный,8-контактное,5,1,,1,2,2,1,1,3,0,0,0,0,0,0,2,1,0,0,0,0,4,0,1,0,1,1,0,0,0,1,244,244
359,ASUS-PRIME-Z270-P,Asus PRIME Z270-P,8000,https://www.e-katalog.ru/ASUS-PRIME-Z270-P.htm,игровая для разгона (overclocking),62,Intel LGA 1151,ATX,Intel Z270,0,1,0,1,2,3866.0,64.0,1,7.1,0,4,2,1.0,1,3,4,0,0,2,0,2,4,0,0,0,0,2,7.0,24-контактный,8-контактное,3,0,"/ RAID 0, RAID 1, RAID 5, RAID 10 /",0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,4,0,0,0,2,1,1,1,0,0,305,226
542,ASUS-ROG-MAXIMUS-Z690-APEX,Asus ROG MAXIMUS Z690 APEX,58200,https://www.e-katalog.ru/ASUS-ROG-MAXIMUS-Z690...,игровая для разгона (overclocking),3,Intel LGA 1700,ATX,Intel Z690,0,0,1,1,2,6600.0,64.0,0,7.1,0,6,5,2.5,1,5,1,1,0,2,0,2,4,5,0,0,1,2,24.0,24-контактный,8+8-контактное,6,0,,1,3,1,2,1,3,1,1,0,0,0,1,0,2,0,0,1,0,2,0,4,0,1,1,0,0,0,0,305,244
185,ASUS-P11C-X,Asus P11C-X (90SB06Q0-M0UAY0),14460,https://www.e-katalog.ru/ASUS-P11C-X.htm,для сервера,36,Intel LGA 1151 v2,ATX,Intel C242,0,1,0,1,2,2666.0,64.0,0,0.0,0,6,2,1.0,2,3,0,0,0,1,2,2,2,2,0,0,0,1,NaN,24-контактный,8-контактное,6,0,"/ RAID 0, 1, 5, 10 /",0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,0,0,0,4,0,0,0,0,1,1,1,0,0,305,244


<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 0 to 616
Data columns (total 72 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 589 non-null    object 
 1   name               589 non-null    object 
 2   price              589 non-null    int64  
 3   link               589 non-null    object 
 4   direction          589 non-null    object 
 5   age                589 non-null    int64  
 6   socket             589 non-null    object 
 7   form               589 non-null    object 
 8   chipset            589 non-null    object 
 9   ddr3               589 non-null    int64  
 10  ddr4               589 non-null    int64  
 11  ddr5               589 non-null    int64  
 12  dimm               589 non-null    int64  
 13  ram_channels       589 non-null    int64  
 14  max_ram_freq       582 non-null    float64
 15  max_ram_vol        588 non-null    float64
 16  dvi                589 non

In [10]:
video = pd.read_csv(path + 'data/raw_data/video.csv')
video.dropna(subset = ['Модель GPU','Потоковых процессоров'],inplace=True)  

video = video[['id', 'name', 'price', 'link', 'Подключение', 'Модель GPU',
'Объем памяти', 'Тип памяти', 'Разрядность шины','Частота работы GPU',
'Частота работы памяти', 'Техпроцесс', 'Макс. разрешение',
'DisplayPort', 'Версия DisplayPort', 'USB C', 'Версия DirectX',
'Версия OpenGL', 'Потоковых процессоров',
'Макс. подключаемых мониторов', 'Охлаждение', 'Кол-во вентиляторов',
'Потребляемая мощность', 'Дополнительное питание',
'Рекомендуемая мощность БП от', 'Занимаемых слотов', 'Длина видеокарты',
'Дата добавления на E-Katalog', 'Тест Passmark G3D Mark', 'HDMI',
'Текстурных блоков', 'miniDisplayPort',
'Синхронизация подсветки', 'VGA', 'DVI-I',
'DVI-D']]

video.columns = ['id', 'name', 'price', 'link', 'connection', 'gpu', 'mem_vol',
'mem_type', 'bus','gpu_freq','mem_freq','techprocess','max_res', 'displayport',
'displayport_ver', 'usb_c', 'directX','openGL','processors','max_monitors',
'cool', 'vent','power', 'add_power','supply_power','slots', 'length','age',
'passmark','hdmi','text_blocks','miniDisplayPort','light_sync',
'vga', 'dvi-i','dvi-d' ]

age(video)

for c in ['connection']:
  video[c] = video[c].apply(lambda x:x.split(' /')[0] if x == x else x)

for c in ['power','supply_power','passmark','text_blocks',
          'gpu_freq','techprocess']:
  smart_fillna(video,c,'gpu')

smart_fillna(video,'length','slots')


to_number(video,[['mem_vol',0],['bus',None],['gpu_freq',None],['mem_freq',None],
                 ['techprocess',None],['displayport',0],['directX',0], ['vent',0], 
                 ['power',None],['supply_power',None],['length',None],
                 ['passmark',None],['hdmi',0],['miniDisplayPort',0]
                 ])

to_bin(video,['usb_c','light_sync','vga','dvi-i','dvi-d'])

video.max_res = video.max_res.apply(lambda x:int(x.split('x')[0] if x == x else -1) )
video.displayport_ver.fillna('no',inplace=True)
video.openGL.fillna(0,inplace=True)
video.processors = video.processors.apply(int)
video.max_monitors = video.max_monitors.fillna(0).apply(int)
video.cool.fillna('no',inplace=True)
video.add_power.fillna('no',inplace=True)
video.slots.fillna(video.slots.median(),inplace=True)
video.to_csv(path + 'data/datasets/video.csv',index = False)

display(video.sample(5))
video.info()

,id,name,price,link,connection,gpu,mem_vol,mem_type,bus,gpu_freq,mem_freq,techprocess,max_res,displayport,displayport_ver,usb_c,directX,openGL,processors,max_monitors,cool,vent,power,add_power,supply_power,slots,length,age,passmark,hdmi,text_blocks,miniDisplayPort,light_sync,vga,dvi-i,dvi-d
182,PNY-QUADRO-P400,PNY Quadro P400 (VCQP400-PB),12733,https://www.e-katalog.ru/PNY-QUADRO-P400.htm,PCI-E v3.0,NVIDIA Quadro P400,2.0,GDDR5,64,1070.0,7000.0,14.0,5120,0,no,0,12.0,4.5,256,3,активное (кулер),1,30.0,no,NaN,1.0,145.0,58,NaN,0,16.0,0,0,0,0,0
208,ASUS-GEFORCE-GTX-1050-TI-CERBERUS-GTX1050TI-O4G,Asus GeForce GTX 1050 Ti CERBERUS-GTX1050TI-O4G,24800,https://www.e-katalog.ru/ASUS-GEFORCE-GTX-1050...,PCI-E v3.0,NVIDIA GeForce GTX 1050 Ti,4.0,GDDR5,128,1366.0,7008.0,14.0,7680,1,no,0,12.0,4.5,768,3,активное (кулер),2,75.0,no,300.0,2.0,203.0,50,5998.0,1,48.0,0,0,0,0,1
80,ASUS-GEFORCE-GTX-1650-PHOENIX,Asus GeForce GTX 1650 PHOENIX (PH-GTX1650-4G),37198,https://www.e-katalog.ru/ASUS-GEFORCE-GTX-1650...,PCI-E v3.0,NVIDIA GeForce GTX 1650,4.0,GDDR5,128,1695.0,8000.0,12.0,7680,1,v.1.4,0,12.0,4.6,896,3,активное (кулер),1,75.0,no,300.0,2.0,190.0,34,5825.0,1,56.0,0,0,0,0,1
192,SAPPHIRE-RADEON-RX-6600-PULSE,Sapphire Radeon RX 6600 PULSE (11310-01-20G),56900,https://www.e-katalog.ru/SAPPHIRE-RADEON-RX-66...,PCI-E v4.0,AMD Radeon RX 6600,8.0,GDDR6,128,2491.0,14000.0,7.0,7680,3,v.1.4,0,12.0,4.6,1792,4,активное (кулер),2,140.0,8 pin,500.0,2.0,193.0,4,19008.0,1,NaN,0,0,0,0,0
456,INNO3D-GEFORCE-GTX-1660-SUPER-TWIN-X2,INNO3D GeForce GTX 1660 SUPER TWIN X2 (N166S2...,73000,https://www.e-katalog.ru/INNO3D-GEFORCE-GTX-16...,PCI-E v3.0,NVIDIA GeForce GTX 1660 SUPER,6.0,GDDR6,192,1785.0,14000.0,12.0,7680,3,v.1.4,0,12.0,4.6,1408,4,активное (кулер),2,125.0,8 pin,450.0,2.0,220.0,28,12677.0,1,88.0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 475 entries, 0 to 483
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               475 non-null    object 
 1   name             475 non-null    object 
 2   price            475 non-null    int64  
 3   link             475 non-null    object 
 4   connection       475 non-null    object 
 5   gpu              475 non-null    object 
 6   mem_vol          475 non-null    float64
 7   mem_type         475 non-null    object 
 8   bus              475 non-null    int64  
 9   gpu_freq         453 non-null    float64
 10  mem_freq         450 non-null    float64
 11  techprocess      468 non-null    float64
 12  max_res          475 non-null    int64  
 13  displayport      475 non-null    int64  
 14  displayport_ver  475 non-null    object 
 15  usb_c            475 non-null    int64  
 16  directX          475 non-null    float64
 17  openGL          

In [11]:
ssd = pd.read_csv(path + 'data/raw_data/ssd.csv')
ssd.dropna(subset = ['Форм-фактор','Объем','Тип'],inplace=True)  
ssd = ssd[['id', 'name', 'price', 'link', 'Тип', 'Объем', 'Форм-фактор', 'Разъем',
       'Тип памяти', 'Внешняя скорость записи', 'Внешняя скорость считывания',
       'Ударостойкость при работе', 'Наработка на отказ', 'IOPS записи',
       'IOPS считывания', 'TBW', 'DWPD', 'Гарантия производителя',
       'Дата добавления на E-Katalog', 'Интерфейс M.2', 'Контроллер', 'NVMe',
       'Размеры', 'Назначение']]

ssd.columns = ['id', 'name', 'price', 'link', 'type', 'volume', 'form', 'connector',
       'memory', 'write', 'read','resist', 'mtbf', 'iops_wr',
       'iops_r', 'tbw', 'dwpd', 'warranty','age', 'm.2', 'controller', 'nvme',
       'dimensions', 'server']

age(ssd)

for c in ['connector','memory','m.2',
          'controller']:
  ssd[c] = ssd[c].apply(lambda x:x.split(' /')[0] if x == x else x)

for c in['connector','memory','m.2',
         'controller','nvme']:
  ssd[c].fillna('no', inplace = True)


to_number(ssd,[['volume'],['write',None],['read',None],['resist',None],
               ['mtbf',None],['iops_wr',None],['iops_r',None],['tbw',None],
               ['dwpd',None],['warranty',None]
               ])

extract_dimensions(ssd)
to_bin(ssd,['server'])
ssd.to_csv(path + 'data/datasets/ssd.csv',index = False)

display(ssd.sample(5))
ssd.info()

,id,name,price,link,type,volume,form,connector,memory,write,read,resist,mtbf,iops_wr,iops_r,tbw,dwpd,warranty,age,m.2,controller,nvme,server,height,width
312,SILICON-POWER-SP128GBSS3A55S25,Silicon Power Ace A55 SP128GBSS3A55S25 128 ГБ,1570,https://www.e-katalog.ru/SILICON-POWER-SP128GB...,внутренний,128,"2.5""",SATA 3,3D TLC NAND,420.0,550.0,1500.0,1.5,NaN,NaN,NaN,NaN,3.0,48.0,no,Silicon Motion SM2258XT,no,0,100.0,70.0
547,TRANSCEND-TS120GMTS420S,Transcend 420S TS120GMTS420S 120 ГБ,1690,https://www.e-katalog.ru/TRANSCEND-TS120GMTS42...,внутренний,120,M.2,no,3D TLC NAND,480.0,520.0,NaN,1.0,75.0,40.0,NaN,NaN,3.0,56.0,SATA 3,no,no,0,22.0,42.0
126,WD-WDS120G2G0A,WD Green SSD WDS120G2G0A 120 ГБ 1 млн. ч,2304,https://www.e-katalog.ru/WD-WDS120G2G0A.htm,внутренний,120,"2.5""",SATA 3,TLC,NaN,545.0,1500.0,1.0,37.0,63.0,40.0,0.31,3.0,48.0,no,Silicon Motion SM2258XT,no,0,100.0,70.0
1106,ACER-SA100-480GB,Acer SA100 SA100-480GB 480 ГБ (BL.9BWWA.103),4710,https://www.e-katalog.ru/ACER-SA100-480GB.htm,внутренний,480,"2.5""",SATA 3,3D TLC NAND,493.0,560.0,NaN,NaN,71.0,82.0,150.0,0.30,3.0,9.0,no,no,no,0,100.0,70.0
800,ACER-FA100-512GB,Acer FA100 M.2 FA100-512GB 512 ГБ,5400,https://www.e-katalog.ru/ACER-FA100-512GB.htm,внутренний,512,M.2,no,3D TLC NAND,2200.0,3200.0,1500.0,2.0,190.0,296.0,300.0,0.30,5.0,9.0,PCI-E 3.0 4x,no,/ 1.4 /,0,22.0,80.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1474 entries, 0 to 1521
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1474 non-null   object 
 1   name        1474 non-null   object 
 2   price       1474 non-null   int64  
 3   link        1474 non-null   object 
 4   type        1474 non-null   object 
 5   volume      1474 non-null   int64  
 6   form        1474 non-null   object 
 7   connector   1474 non-null   object 
 8   memory      1474 non-null   object 
 9   write       1326 non-null   float64
 10  read        1352 non-null   float64
 11  resist      938 non-null    float64
 12  mtbf        1042 non-null   float64
 13  iops_wr     756 non-null    float64
 14  iops_r      744 non-null    float64
 15  tbw         764 non-null    float64
 16  dwpd        745 non-null    float64
 17  warranty    1218 non-null   float64
 18  age         1473 non-null   float64
 19  m.2         1474 non-null  

In [12]:
ram = pd.read_csv(path + 'data/raw_data/ram.csv')
ram.dropna(subset = ['Объем памяти комплекта','Тип памяти','Форм-фактор памяти',
                     'Пропускная способность','CAS-латентность',
                     'Тактовая частота','Рабочее напряжение',
                     'Кол-во планок в комплекте','Дата добавления на E-Katalog'],inplace=True)  

ram = ram[(ram['Тип памяти'] == 'DDR3') | (ram['Тип памяти'] == 'DDR4')]

ram = ram[['id', 'name', 'price', 'link', 'Объем памяти комплекта',
       'Кол-во планок в комплекте', 'Форм-фактор памяти', 'Тип памяти',
       'Тактовая частота', 'Пропускная способность', 'CAS-латентность',
       'Рабочее напряжение', 'Тип охлаждения','Дополнительно',
       'Синхронизация подсветки','Дата добавления на E-Katalog', 'Ранг памяти']]

ram.columns = ['id', 'name', 'price', 'link', 'volume','n_planks', 'dimm',
               'ddr3','freq', 'bandwidth', 'cas','voltage', 'cooling',
               'overclock','lighting','age', 'rank']

age(ram)

to_number(ram,[['volume'],['n_planks'],['freq'],['bandwidth'],
               ['voltage']
               ])

ram.cas = ram.cas.apply(lambda x:int(x.replace('CL','')))

ram.overclock = ram.overclock.apply(
                lambda x: 1 if x==x and ('overclock' in x) else 0)
to_bin(ram,['lighting'])
ram['rank'] = ram['rank'].apply(lambda x:x.split(' /')[0] if x == x else 1)
ram['rank'] = ram['rank'].apply(lambda x:1 if (type(x) == str) and ('одно' in x) else x)
ram['rank'] = ram['rank'].apply(lambda x:2 if (type(x) == str) and ('двух' in x) else x)
ram['rank'] = ram['rank'].apply(lambda x:4 if (type(x) == str) and ('четырех' in x) else x)

ram.dimm = ram.dimm.apply(lambda x: 1 if x == 'DIMM' else 0)
ram.ddr3 = ram.ddr3.apply(lambda x: 1 if x == 'DDR3' else 0)
ram.cooling = ram.cooling.apply(lambda x: 0 if x == 'без охлаждения' else 1)

ram.to_csv(path + 'data/datasets/ram.csv',index = False)

display(ram.sample(5))
ram.info()

,id,name,price,link,volume,n_planks,dimm,ddr3,freq,bandwidth,cas,voltage,cooling,overclock,lighting,age,rank
937,FUJITSU-S26361-F4083-L332,Fujitsu DDR4 1x32Gb S26361-F4083-L332,16990,https://www.e-katalog.ru/FUJITSU-S26361-F4083-...,32,1,1,0,2933,23400,21,1.20,0,0,0,12,1
1759,E2E4-D4-3200-CL22-2X8G,E2E4 DDR4 2x8Gb D4-3200-CL22-2x8G,7830,https://www.e-katalog.ru/E2E4-D4-3200-CL22-2X8...,16,2,1,0,3200,25600,22,1.20,0,0,0,6,1
348,CRUCIAL-BL2K8G26C16U4B,Crucial Ballistix DDR4 2x8Gb BL2K8G26C16U4B,6900,https://www.e-katalog.ru/CRUCIAL-BL2K8G26C16U4...,16,2,1,0,2666,21300,16,1.35,1,1,0,24,1
595,CORSAIR-CMV8GX4M1A2666C18,Corsair ValueSelect DDR4 1x8Gb CMV8GX4M1A2666C18,3290,https://www.e-katalog.ru/CORSAIR-CMV8GX4M1A266...,8,1,1,0,2666,21300,18,1.20,1,0,0,49,1
856,GEIL-GOG416GB4400C18ADC,Geil ORION DDR4 2x8Gb GOG416GB4400C18ADC,11190,https://www.e-katalog.ru/GEIL-GOG416GB4400C18A...,16,2,1,0,4400,35200,22,1.40,1,1,0,9,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2023 entries, 0 to 2126
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2023 non-null   object 
 1   name       2023 non-null   object 
 2   price      2023 non-null   int64  
 3   link       2023 non-null   object 
 4   volume     2023 non-null   int64  
 5   n_planks   2023 non-null   int64  
 6   dimm       2023 non-null   int64  
 7   ddr3       2023 non-null   int64  
 8   freq       2023 non-null   int64  
 9   bandwidth  2023 non-null   int64  
 10  cas        2023 non-null   int64  
 11  voltage    2023 non-null   float64
 12  cooling    2023 non-null   int64  
 13  overclock  2023 non-null   int64  
 14  lighting   2023 non-null   int64  
 15  age        2023 non-null   int64  
 16  rank       2023 non-null   int64  
dtypes: float64(1), int64(13), object(3)
memory usage: 284.5+ KB


In [13]:
power = pd.read_csv(path + 'data/raw_data/power.csv')
power = power[['id', 'name', 'price', 'link', 'Мощность', 'Форм-фактор',
       'Система охлаждения', 'Диаметр вентилятора',
       'Сертификат', 'Стандарт ATX 12В v.', 
       'Питание MB/CPU', 'SATA', 'MOLEX','PCI-E 8pin (6+2)', 'Система кабелей',
       'Габариты (ВхШхГ)', 'Дата добавления на E-Katalog',
       'Тип PFC','КПД', 'Тип подшипника', 'Стандарт EPS 12В v.', 
       '+3.3V', '+5V', '+12V1', '-12V', '+5Vsb', 'Мощность +12V',
       'Мощность +3.3V +5V', 'Гарантия производителя']]

power.columns= ['id', 'name', 'price', 'link', 'power', 'form',
       'cool', 'vent','sertificate', 'atx_std', 'mb/cpu', 'sata', 'molex','pci_e',
       'cable_sys','dimensions','age','active_pfc','efficiency', 'bearing', 'eps', 
       '+3.3V', '+5V', '+12V1', '-12V', '+5Vsb', '+12V_watt',
       '+3.3V_+5V_watt', 'warranty']

power.dropna(subset = ['power','form','cool','mb/cpu',
                       'cable_sys'],inplace=True)  
age(power)
power.active_pfc = power.active_pfc.apply(
               lambda x: 1 if x==x and ('актив' in x) else 0)

to_number(power,[['power'],['efficiency',None],['vent',0],['sata',0],
                 ['molex',0],['pci_e',0],['+3.3V',None], ['+5V',None], 
                 ['+12V1',None], ['-12V',None], ['+5Vsb',None], 
                 ['+12V_watt',None],['+3.3V_+5V_watt',None], ['warranty',None],
                 
                 ])
power.cool = power.cool.apply(lambda x: 1 if '1' in x else(2 if '2' in x else 0))

power.sertificate = power.sertificate.apply(
                     lambda x:x.split(' /')[0] if x == x else 'без 80+')
extract_dimensions(power)
power.to_csv(path + 'data/datasets/power.csv',index = False)
display(power.sample(5))
power.info()

,id,name,price,link,power,form,cool,vent,sertificate,atx_std,mb/cpu,sata,molex,pci_e,cable_sys,age,active_pfc,efficiency,bearing,eps,+3.3V,+5V,+12V1,-12V,+5Vsb,+12V_watt,+3.3V_+5V_watt,warranty,height,width,deep
457,AEROCOOL-VX-PLUS-550W,Aerocool Value Plus VX Plus 550W,2250,https://www.e-katalog.ru/AEROCOOL-VX-PLUS-550W...,550,ATX,1,120,без 80+,2.30,24+8 (4+4) pin,3,3,1,не модульная,42.0,0,82.0,скольжения,NaN,18.0,16.0,42.0,0.3,2.5,504.0,120.0,2.0,NaN,NaN,NaN
456,EXEGATE-ATX-UN700,ExeGate UN ATX-UN700,2525,https://www.e-katalog.ru/EXEGATE-ATX-UN700.htm,700,ATX,1,120,без 80+,2.30,24+4 pin,3,2,1,не модульная,56.0,0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,150.0,140.0
586,COOLER-MASTER-MPE-6001-ACABN,Cooler Master Elite V4 MPE-6001-ACABN,3500,https://www.e-katalog.ru/COOLER-MASTER-MPE-600...,600,ATX,1,120,80+,2.41,24+8 (4+4) pin,5,3,2,не модульная,14.0,1,85.0,NaN,NaN,20.0,20.0,46.0,0.3,3.0,552.0,110.0,3.0,86.0,150.0,140.0
691,COUGAR-XTC650,Cougar XTC XTC650 (31XC650.0001P),3174,https://www.e-katalog.ru/COUGAR-XTC650.htm,650,ATX,1,120,80+,NaN,24+8 (4+4) pin,6,2,2,не модульная,3.0,1,86.0,NaN,NaN,18.0,20.0,52.0,0.3,3.0,624.0,130.0,NaN,86.0,140.0,150.0
29,EXEGATE-UNS700,ExeGate UNS UNS700,2410,https://www.e-katalog.ru/EXEGATE-UNS700.htm,700,ATX,1,120,без 80+,2.30,24+4 pin,3,2,1,не модульная,56.0,0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,86.0,150.0,140.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 963 entries, 0 to 963
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              963 non-null    object 
 1   name            963 non-null    object 
 2   price           963 non-null    int64  
 3   link            963 non-null    object 
 4   power           963 non-null    int64  
 5   form            963 non-null    object 
 6   cool            963 non-null    int64  
 7   vent            963 non-null    int64  
 8   sertificate     963 non-null    object 
 9   atx_std         693 non-null    float64
 10  mb/cpu          963 non-null    object 
 11  sata            963 non-null    int64  
 12  molex           963 non-null    int64  
 13  pci_e           963 non-null    int64  
 14  cable_sys       963 non-null    object 
 15  age             962 non-null    float64
 16  active_pfc      963 non-null    int64  
 17  efficiency      773 non-null    flo

In [14]:
hdd = pd.read_csv(path + 'data/raw_data/hdd.csv')
hdd = hdd[hdd['Тип накопителя'] == 'HDD']
hdd = hdd[['id', 'name', 'price', 'link', 'Исполнение', 'Назначение', 'Объем',
       'Форм-фактор', 'Интерфейсы подключения', 'Объем буфера обмена',
       'Частота вращения шпинделя', 'Среднее время поиска',
       'Ударостойкость при работе', 'Уровень шума при чтении',
       'Уровень шума в режиме ожидания', 'Размеры', 'Вес',
       'Дата добавления на E-Katalog','Гарантия производителя',
       'Потребляемая мощность при работе','Потребляемая мощность при ожидании']]
hdd.columns = ['id', 'name', 'price', 'link', 'inner','purpose', 'volume',
       'form','interface',  'buffer','spindel', 'search_time','resistance',
       'noise_read','noise_wait', 'dimensions', 'weight', 'age',
       'warranty', 'consume_work','consume_wait']

hdd.dropna(subset = ['inner','purpose','volume','form','interface'],inplace = True)
age(hdd)
hdd.inner = hdd.inner.apply(lambda x: 1 if 'внеш' in x else 0)
hdd.spindel = hdd.spindel.apply(
    lambda x: '7200 ' if x == x and (('7200' in x) or ('измен' in x)) else x)

to_number(hdd, [['volume'],['form'],['buffer',None],['spindel',None],
                ['search_time',None],['resistance',None],['noise_read',None],
                ['noise_wait',None],['weight',None],['warranty',None],
                ['consume_work',None], ['consume_wait',None]
                ])

for i in ['IDE','SAS','SATA','SATA 2','SATA 3','USB 3.2 gen1','USB 3.2 gen2',
'USB C 3.2 gen1','USB C 3.2 gen2','Thunderbolt v3','USB 2.0',  
'micro-USB','IEEE 1394','Fibre Channel']:
  hdd[i] = hdd.interface.apply(lambda x: 1 if i in x else 0)

hdd.drop('interface',axis=1,inplace = True)
extract_dimensions(hdd)
hdd.to_csv(path + 'data/datasets/hdd.csv',index = False)


display(hdd.sample(5))
hdd.info()

,id,name,price,link,inner,purpose,volume,form,buffer,spindel,search_time,resistance,noise_read,noise_wait,weight,age,warranty,consume_work,consume_wait,IDE,SAS,SATA,SATA 2,SATA 3,USB 3.2 gen1,USB 3.2 gen2,USB C 3.2 gen1,USB C 3.2 gen2,Thunderbolt v3,USB 2.0,micro-USB,IEEE 1394,Fibre Channel,height,width,deep
754,DELL-400-BEGP,Dell SAS 7.2K 400-BEGP 4 ТБ BEGP,18335,https://www.e-katalog.ru/DELL-400-BEGP.htm,0,для сервера,4000,3.5,NaN,7200.0,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1195,LENOVO-4XB0G88732,"Lenovo ThinkServer HDD 2.5"" 4XB0G88732 300 ГБ",25375,https://www.e-katalog.ru/LENOVO-4XB0G88732.htm,0,для сервера,300,2.5,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,66,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
86,SEAGATE-ST6000VN0001,Seagate Enterprise NAS HDD ST6000VN0001 6 ТБ,19999,https://www.e-katalog.ru/SEAGATE-ST6000VN0001.htm,0,для сервера,6000,3.5,128.0,7200.0,NaN,250.0,NaN,NaN,780.0,81,5.0,9.0,6.9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,147.0,101.0,85.0
471,LENOVO-7XB7A00069,"Lenovo SAS 10K Hot Swapp 2.5"" 7XB7A00069 2.4 ТБ",29972,https://www.e-katalog.ru/LENOVO-7XB7A00069.htm,0,для сервера,2400,2.5,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1069,IBM-00Y2507,"IBM V3700 2.5"" 00Y2507 1.2 ТБ",20285,https://www.e-katalog.ru/IBM-00Y2507.htm,0,для сервера,1200,2.5,16.0,10000.0,NaN,NaN,NaN,NaN,NaN,76,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 0 to 1751
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1731 non-null   object 
 1   name            1731 non-null   object 
 2   price           1731 non-null   int64  
 3   link            1731 non-null   object 
 4   inner           1731 non-null   int64  
 5   purpose         1731 non-null   object 
 6   volume          1731 non-null   int64  
 7   form            1731 non-null   float64
 8   buffer          974 non-null    float64
 9   spindel         1628 non-null   float64
 10  search_time     585 non-null    float64
 11  resistance      604 non-null    float64
 12  noise_read      404 non-null    float64
 13  noise_wait      408 non-null    float64
 14  weight          767 non-null    float64
 15  age             1731 non-null   int64  
 16  warranty        918 non-null    float64
 17  consume_work    687 non-null    f

In [15]:
cool = pd.read_csv(path + 'data/raw_data/cool.csv')
cool = cool[['id', 'name', 'price', 'link', 'Назначение', 'Тип', 
       'Вентиляторов', 'Тип крепления', 'Диаметр вентилятора',
       'Тип подшипника', 'Минимальные обороты', 'Максимальные обороты',
       'Регулятор оборотов', 'Макс. воздушный поток', 'Выдув воздушного потока',
       'Наработка на отказ', 'Питание', 'Уровень шума', 'Габариты', 
       'Дата добавления на E-Katalog', 'Материал радиатора', 'Socket', 
       'Цвет подсветки','Тепловых трубок'
       ]]

cool.columns = ['id', 'name', 'price', 'link', 'purpose', 'type', 
       'n_vents', 'tier', 'diameter','bearing', 'min_turn', 'max_turn',
       'reg_turn', 'max_airflow','flow_direction','mtlb', 'power','noise', 'dimensions', 
       'age','material_rad', 'socket', 'lightening','pipes']

cool = cool[(cool.purpose == 'корпус') | (cool.purpose =='процессор')]
cool = cool[(cool['type'] == 'вентилятор') | (cool['type'] =='активный кулер')]
cool.dropna(subset = ['diameter','bearing','max_turn'],inplace=True)
age(cool)
extract_dimensions(cool)

to_number(cool,[['n_vents',1],['min_turn',0],['max_turn'],['diameter'],
                ['max_airflow',None],['mtlb',None],['noise',None],
                ['pipes',0]              
                ])

for c in ['bearing','tier','reg_turn','power','material_rad']:
 cool[c] = cool[c].apply(lambda x: x.split()[0] if x==x else x)   

cool.min_turn[cool.min_turn == 0] = \
cool.min_turn[cool.min_turn == 0].combine(cool.max_turn[cool.min_turn == 0],max)

cool.to_csv(path + 'data/datasets/cool.csv',index = False)
display(cool.sample(5))
cool.info()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,name,price,link,purpose,type,n_vents,tier,diameter,bearing,min_turn,max_turn,reg_turn,max_airflow,flow_direction,mtlb,power,noise,age,material_rad,socket,lightening,pipes,height,width
181,NOISEBLOCKER-IP55-SERIE-5010-30-12,Noiseblocker IP55 Serie 5010-30-12,1150,https://www.e-katalog.ru/NOISEBLOCKER-IP55-SER...,корпус,вентилятор,1,болты,50,качения,3000,3000,отсутствует,5.18,NaN,NaN,3-pin,17.0,12,NaN,NaN,NaN,0,50.0,50.0
1394,BE-QUIET-PURE-WINGS-2-140-HIGH-SPEED,Be quiet Pure Wings 2 140 High-Speed (BL082),970,https://www.e-katalog.ru/BE-QUIET-PURE-WINGS-2...,корпус,вентилятор,1,болты,140,скольжения,1600,1600,отсутствует,94.20,NaN,80.0,3-pin,37.0,30,NaN,NaN,NaN,0,140.0,140.0
1386,THERMALRIGHT-SILVERARROW-ITX,Thermalright SilverArrow ITX,7100,https://www.e-katalog.ru/THERMALRIGHT-SILVERAR...,процессор,активный кулер,1,двусторонний,140,скольжения,1300,1300,авто,61.70,вбок (рассеивание),50.0,4-pin,25.0,73,алюминий/медь,AMD AM2/AM3/FM1/FM2Intel 775Intel 1150Intel 11...,NaN,6,154.0,103.0
389,NOCTUA-NF-F12-INDUSTRIALPPC-2000,Noctua NF-F12 industrialPPC-2000,2090,https://www.e-katalog.ru/NOCTUA-NF-F12-INDUSTR...,корпус,вентилятор,1,болты,120,гидродинамический,2000,2000,отсутствует,71.74,NaN,150.0,3-pin,30.0,48,NaN,NaN,NaN,0,120.0,120.0
533,XILENCE-RED-WING-80,Xilence RED WING 80,210,https://www.e-katalog.ru/XILENCE-RED-WING-80.htm,корпус,вентилятор,1,болты,80,качения,1800,1800,отсутствует,19.60,NaN,NaN,3-pin,19.0,105,NaN,NaN,NaN,0,80.0,80.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223 entries, 0 to 1487
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1223 non-null   object 
 1   name            1223 non-null   object 
 2   price           1223 non-null   int64  
 3   link            1223 non-null   object 
 4   purpose         1223 non-null   object 
 5   type            1223 non-null   object 
 6   n_vents         1223 non-null   int64  
 7   tier            1223 non-null   object 
 8   diameter        1223 non-null   int64  
 9   bearing         1223 non-null   object 
 10  min_turn        1223 non-null   int64  
 11  max_turn        1223 non-null   int64  
 12  reg_turn        1223 non-null   object 
 13  max_airflow     1102 non-null   float64
 14  flow_direction  439 non-null    object 
 15  mtlb            541 non-null    float64
 16  power           1223 non-null   object 
 17  noise           1113 non-null   f

In [16]:
case = pd.read_csv(path + 'data/raw_data/case.csv')
for text in ['окно на боковой панели','скрытая протяжка проводов',
             'окно установки СО для процессора','закаленное стекло']:
  case[text] =case['Дополнительно*'].apply(lambda x: 1 if x== x and text in x else 0)

case = case[['id', 'name', 'price', 'link','Дата добавления на E-Katalog','Форм-фактор',
             'Тип материнской платы', 'Форм-фактор БП', 'Габариты (ВхШхГ)','Длина видеокарты, до', 
             'Высота кулера, до','Вес','Материал', 'Толщина боковых стенок','Расположение БП', 
             'Отсеков 5,25"', 'Внутренних отсеков 3,5"','Внешних отсеков 3,5"','Внутренних отсеков 2,5"',
             'Отверстий под слоты расширения','Мест для вентиляторов сзади', 'Мест для вентиляторов сбоку',
             'Мест для вентиляторов спереди','Мест для вентиляторов сверху','Мест для вентиляторов снизу',
             'Мест для вентиляторов','Установленных вентиляторов','Расположение', 'USB 2.0','USB 3.2 gen1',
             'USB 3.2 gen2','USB C 3.2 gen1','USB C 3.2 gen2','По направлению', 'Тип подсветки',
             'Мест для СВО','Мощность комплектного БП','окно на боковой панели','скрытая протяжка проводов',
             'окно установки СО для процессора','закаленное стекло']]

case.columns = ['id', 'name', 'price', 'link','age', 'form','mb_type', 'power_form',
       'dimensions','v_card_length', 'cooler_height', 'weight','mateial',
       'sidewall_thickness','PS_location', '5,25"_cell', '3,5"_cell','outer_3,5"',
       '2,5"_cell','expansion_slots','vents_back', 'vents_side','vents_front',
       'vents_up', 'vents_down', 'all_vents_places','mounted_vents','location',
       'usb_2','usb_3_gen1','usb_3_gen2','usb_c_gen1','usb_c_gen2','gaming', 
       'light', 'water','power','side_window','hidden_wire',
       'cool_window','strained_glass']

case.dropna(subset = ['mb_type','form','power_form',
                      'PS_location'],inplace=True)
age(case)

for c in ['form','mateial','location']:
  case[c] = case[c].apply(lambda x:x.split(' /')[0] if x == x else x)
extract_dimensions(case)
to_number(case,[['v_card_length',None],['cooler_height',None],['weight',None],
                ['sidewall_thickness',None],['5,25"_cell',0], ['3,5"_cell',0],
                ['2,5"_cell',0],['expansion_slots',0],['all_vents_places',0],   
                ['outer_3,5"',0],['mounted_vents',0],['usb_2',0],
                ['usb_3_gen1',0],['usb_3_gen2',0],['usb_c_gen1',0],
                ['usb_c_gen2',0],['water',0],['power',0]
                ])

for c in ['vents_back', 'vents_side','vents_front',
       'vents_up', 'vents_down']:
 case[c] = case[c].str.findall(r'\d\D\d+').apply(full_diameter)
case['usb_3'] = case.usb_3_gen1 + case.usb_3_gen2
case['usb_c'] = case.usb_c_gen1 + case.usb_c_gen2

case.drop(['usb_3_gen1','usb_3_gen2',
           'usb_c_gen1','usb_c_gen2'],axis = 1,inplace = True)
to_bin(case,['gaming','light'])

case.to_csv(path + 'data/datasets/case.csv',index = False)

display(case.sample(5))
case.info()

,id,name,price,link,age,form,mb_type,power_form,v_card_length,cooler_height,weight,mateial,sidewall_thickness,PS_location,"5,25""_cell","3,5""_cell","outer_3,5""","2,5""_cell",expansion_slots,vents_back,vents_side,vents_front,vents_up,vents_down,all_vents_places,mounted_vents,location,usb_2,gaming,light,water,power,side_window,hidden_wire,cool_window,strained_glass,height,width,deep,usb_3,usb_c
1156,ZALMAN-N5-OF,Zalman N5 OF черный,2970,https://www.e-katalog.ru/ZALMAN-N5-OF.htm,21,Midi Tower,ATX,ATX (обычный),365.0,158.0,3.60,сталь,NaN,нижнее,1,2,0,4,7,120,0,360,520,240,8,4,сверху корпуса,2,1,1,2,0,1,1,1,0,450.0,200.0,437.0,1,0
985,ZALMAN-Z8-MS,Zalman Z8 MS черный,5410,https://www.e-katalog.ru/ZALMAN-Z8-MS.htm,9,Midi Tower,E-ATX,ATX (обычный),350.0,160.0,6.30,сталь,NaN,нижнее,0,2,0,4,7,120,0,780,520,0,6,4,сверху корпуса,1,1,1,3,0,1,1,1,0,493.0,212.0,460.0,2,0
367,IN-WIN-ES726-450W,In Win ES726 450W БП 450 Вт черный,3860,https://www.e-katalog.ru/IN-WIN-ES726-450W.htm,12,Mini Tower,micro-ATX,ATX (обычный),350.0,NaN,4.15,сталь,NaN,верхнее,1,2,0,0,4,92,0,92,0,0,2,0,на передней панели,2,0,0,0,450,0,0,0,0,375.0,175.0,415.0,2,0
1171,FRACTAL-DESIGN-DEFINE-XL-R2-FD-CA-DEF-XL-R2-BL,Fractal Design Define XL черный (FD-CA-DEF-XL-...,13470,https://www.e-katalog.ru/FRACTAL-DESIGN-DEFINE...,100,Full Tower,XL-ATX,ATX (обычный),480.0,170.0,16.40,сталь,NaN,нижнее,4,8,0,0,9,260,260,260,520,260,6,3,сверху корпуса,2,1,0,0,0,0,1,1,0,559.0,232.0,560.0,2,0
272,POWERMAN-ES725-400W,Powerman ES725 400W БП 400 Вт черный,2091,https://www.e-katalog.ru/POWERMAN-ES725-400W.htm,46,Mini Tower,micro-ATX,ATX (обычный),350.0,NaN,4.00,сталь,0.5,верхнее,2,2,1,0,4,170,340,170,0,0,4,0,на передней панели,0,0,0,0,400,0,0,0,0,368.0,173.0,365.0,2,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1343 entries, 0 to 1356
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1343 non-null   object 
 1   name                1343 non-null   object 
 2   price               1343 non-null   int64  
 3   link                1343 non-null   object 
 4   age                 1343 non-null   int64  
 5   form                1343 non-null   object 
 6   mb_type             1343 non-null   object 
 7   power_form          1343 non-null   object 
 8   v_card_length       1258 non-null   float64
 9   cooler_height       1240 non-null   float64
 10  weight              1082 non-null   float64
 11  mateial             1343 non-null   object 
 12  sidewall_thickness  678 non-null    float64
 13  PS_location         1343 non-null   object 
 14  5,25"_cell          1343 non-null   int64  
 15  3,5"_cell           1343 non-null   int64  
 16  outer_

In [17]:
note = pd.read_csv(path + 'data/raw_data/notebooks.csv')
note.dropna(subset=['Частота смены кадров'],inplace=True)
note = note[['id', 'name','price', 'link', 'Тип', 'Диагональ экрана', 'Тип матрицы', 'Покрытие экрана',
 'Разрешение дисплея', 'Частота смены кадров', 'Яркость','Серия',  'Кодовое название',
 'Кол-во ядер', 'Кол-во потоков', 'Тактовая частота', 'Частота TurboBoost / TurboCore',
 'Объем кэш памяти 2-го уровня', 'Объем кэш памяти 3-го уровня', 'Тест Passmark CPU Mark',
 'Объем оперативной памяти', 'Максимально устанавливаемый объем', 'Тип памяти', 'Частота памяти',
 'Кол-во слотов', 'Тип видеокарты', 'Серия видеокарты', 'Модель видеокарты', 'Объем видеопамяти',
 'Тип памяти*', 'Тип накопителя', 'Емкость накопителя', 'Интерфейс накопителя M.2',
 'USB 2.0', 'USB 3.2 gen1','USB 3.2 gen2', 'USB C 3.2 gen1', 'USB C 3.2 gen2',
 'Wi-Fi', 'Web-камера', 'Количество динамиков',
  'Емкость батареи','Емкость батареи*', 'Макс. время работы', 
 'Предустановленная ОС',  'Габариты (ШхГхТ)', 'Вес','Дата добавления на E-Katalog', 'Тест 3DMark06',
 'Картридер', 'Дополнительный разъем M.2', 'USB4', 'Интерфейс Thunderbolt', 'Макс. подключаемых мониторов'
# 'Материал корпуса','Bluetooth',
# 'Быстрая зарядка','Кол-во ячеек батареи','Размер накопителя M.2', #'Брендовая акустика', 'Аудиодекодеры','Манипулятор','Напряжение батареи',
# 'Интерфейс доп. разъема M.2', 'Размер доп. накопителя M.2','Стандарт защиты MIL-STD-810', 'Цветовой охват (NTSC)', 'Цветовой охват (DCI P3)',
#  'Цветовой охват (Adobe RGB)', 'Дополнительных клавиш', 'Время отклика',#  'Стекло Gorilla Glass', 'LAN (RJ-45)','Время зарядки',
# 'Синхронизация подсветки', 'Интерфейс разъема M.2',#  'Комплектация', 'USB C 3.2 gen2x2', 'Вес (планшет)', 'Взаимодействие со смартфоном',
#  'Шторка для камеры', 'NFC-чип', 'Подключение док-станции', 'Сенсорный', 'Влагозащита','Модель','Порты подключения',  'Безопасность',
]]
note.columns = ['id', 'name','price', 'link', 'type', 'diagonal', 'matrix', 'screen_coverage',
 'display_res', 'frame_rate', 'bright','series', 'code_name', 'cores', 'threads', 'clock_freq', 'turbo_freq',
 'l2_cashe', 'l3_cashe', 'passmark_test','ram_vol', 'max_ram_vol', 'ddr4', 'ram_freq','ram_slots', 
 'discrete_video', 'video_series', 'video_model', 'video_ram_vol', 'video_ram_type', 'drive_type', 'drive_vol', 'm2_interface',
 'usb_2', 'usb_3_g1','usb_3_g2', 'usb_c_g1', 'usb_c_g2', 'wi_fi', 'camera', 'dynamics', 
 'bat_capacity','bat_capacity*', 'max_work_time','op_sys', 'dimensions', 'weight',
 'age', '3dmark','cardreader','add_m2', 'usb4', 'thunderbolt', 'max_monitors']

extract_dimensions(note)
age(note)
note.turbo_freq.fillna(note.clock_freq,inplace=True)
note.max_monitors.fillna(0,inplace=True)
note.max_ram_vol.fillna(note.ram_vol,inplace=True)
note.bat_capacity.fillna(note['bat_capacity*'],inplace=True)
note.drop('bat_capacity*',axis = 1,inplace = True)
note.dropna(subset = ['diagonal','matrix','screen_coverage','frame_rate','display_res',
                      'cores','clock_freq','ram_vol','ddr4','ram_freq','bat_capacity'],inplace=True)

note.threads.fillna(note.threads.mean(),inplace=True)
note.ram_slots.replace('встроенная + 1 слот',1,inplace=True)
note.ram_slots.replace('встроенная',0,inplace=True)
note.ram_slots = pd.to_numeric(note.ram_slots)
smart_fillna(note,'max_work_time','bat_capacity')
note.thunderbolt = note.thunderbolt.apply(lambda x: x.split(' ')[1] if x==x else x)


to_number(note,[['diagonal',None],['frame_rate',None],['bright',int(note.bright.mode()[0].split('\xa0')[0])],
                ['clock_freq',None],['turbo_freq',None],['l2_cashe',0],['l3_cashe',0],['ram_vol',None],
                ['passmark_test',int(note.passmark_test.mode()[0].split('\xa0')[0])],['max_ram_vol',None],
                ['ram_freq',None],['video_ram_vol',0],['drive_vol',0],['usb_2',0],['usb_3_g1',0],['usb_3_g2',0],
                ['usb_c_g1',0],['usb_c_g2',0],['dynamics',0],['bat_capacity',0],['weight',0],['add_m2',0],
                ['max_work_time',int(note.max_work_time.mode()[0].split('\xa0')[0])],['usb4',0],['thunderbolt',0],
                ['3dmark',int(note['3dmark'].mode()[0].split('\xa0')[0])]
                ])

note.bat_capacity = note.bat_capacity.apply(lambda x: x/1000*12 if x>1000 else x)  

note.ddr4 = note.ddr4.apply(lambda x:1 if 'DDR4' in x else 0)
note.discrete_video = note.discrete_video.apply(lambda x:1 if 'дискретная' in x else 0)
note.screen_coverage = note.screen_coverage.apply(lambda x: x.split(' /')[0])
note.display_res = note.display_res.apply(lambda x: int(x.split('x')[0]))
note.video_model = note.video_model.apply(lambda x: x.split(' ')[0])
note.m2_interface = note.m2_interface.apply(lambda x: x.split(' /')[0] if x==x else x)
note.camera = note.camera.apply(lambda x: 0 if 'отсутствует' in x else int(x.split('x')[0]))

cat_cols = ['matrix','screen_coverage','type','series','code_name','video_series',
            'video_model','video_ram_type','drive_type','m2_interface','wi_fi','op_sys',
            'cardreader']    
for c in cat_cols:
    encoder = ce.PolynomialEncoder()
    X= encoder.fit_transform(note[c],note.price)
    note.drop(c,axis = 1,inplace = True)
    note = note.join(X.drop('intercept',axis=1))
note.to_csv(path + 'data/datasets/notebooks.csv',index = False)

display(note.sample(5))
note.iloc[:,0:40].info()

,id,name,price,link,diagonal,display_res,frame_rate,bright,cores,threads,clock_freq,turbo_freq,l2_cashe,l3_cashe,passmark_test,ram_vol,max_ram_vol,ddr4,ram_freq,ram_slots,discrete_video,video_ram_vol,drive_vol,usb_2,usb_3_g1,usb_3_g2,usb_c_g1,usb_c_g2,camera,dynamics,bat_capacity,max_work_time,weight,age,3dmark,add_m2,usb4,thunderbolt,max_monitors,height,...,video_ram_type_0,video_ram_type_1,video_ram_type_2,drive_type_0,drive_type_1,drive_type_2,drive_type_3,drive_type_4,drive_type_5,drive_type_6,drive_type_7,drive_type_8,m2_interface_0,m2_interface_1,m2_interface_2,m2_interface_3,m2_interface_4,wi_fi_0,wi_fi_1,op_sys_0,op_sys_1,op_sys_2,op_sys_3,op_sys_4,op_sys_5,op_sys_6,op_sys_7,op_sys_8,op_sys_9,op_sys_10,op_sys_11,op_sys_12,op_sys_13,cardreader_0,cardreader_1,cardreader_2,cardreader_3,cardreader_4,cardreader_5,cardreader_6
1844,/ASUS-X712EA-AU363.htm,Asus VivoBook 17 X712EA (X712EA-AU363) (90NB0T...,48990,https://www.e-katalog.ru/ASUS-X712EA-AU363.htm,17.3,1920,60,250,2,4.0,3.0,4.1,2560,6,6482,8,16,1,3200,1,0,0.0,512,2,1,0,1,0,1280,2.0,32.0,8.00,2.30,3,6331,0,0,0,1.0,407,...,-0.670820,0.5,-0.223607,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.119523,-0.436436,0.298142,0.377964,-0.629941,-7.071068e-01,0.408248,-0.358569,0.269869,-0.065179,-0.168703,0.351684,-0.438004,0.425576,-0.345159,0.239071,-0.141982,0.071638,-0.029959,0.009845,-0.002210,-0.540062,0.540062,-0.430820,0.282038,-0.149786,0.061546,-0.017070
3442,/ASUS-K3500PC-L1009T.htm,Asus Vivobook Pro 15 OLED K3500PC (K3500PC-L10...,104999,https://www.e-katalog.ru/ASUS-K3500PC-L1009T.htm,15.6,1920,60,400,4,8.0,3.1,4.4,5120,8,11439,16,16,1,3200,0,1,4.0,512,2,1,0,0,0,1280,2.0,63.0,15.00,1.65,2,10250,0,1,1,2.0,360,...,0.223607,-0.5,-0.670820,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.119523,-0.436436,0.298142,0.377964,-0.629941,-7.071068e-01,0.408248,-0.298807,0.098606,0.175483,-0.341733,0.300960,-0.084231,-0.185507,0.379950,-0.436039,0.371110,-0.248910,0.132074,-0.053325,0.014367,-0.540062,0.540062,-0.430820,0.282038,-0.149786,0.061546,-0.017070
2480,/HP-240G8-2X7R5EA.htm,HP 240 G8 (240G8 2X7R5EA),62900,https://www.e-katalog.ru/HP-240G8-2X7R5EA.htm,14.0,1920,60,250,4,8.0,1.3,3.9,2048,8,8908,8,16,1,2666,2,0,0.0,256,0,2,0,1,0,1280,2.0,43.2,9.25,1.47,12,9208,0,0,0,1.0,324,...,-0.670820,0.5,-0.223607,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.119523,-0.436436,0.298142,0.377964,-0.629941,-7.071068e-01,0.408248,-0.418330,0.472270,-0.456256,0.393641,-0.307724,0.219002,-0.141859,0.083314,-0.044039,0.020706,-0.008499,0.002954,-0.000820,0.000158,-0.231455,-0.231455,0.430820,-0.120873,-0.363766,0.553912,-0.358464
1342,/ASUS-G733QS-HG003T.htm,Asus ROG Strix SCAR 17 G733QS (G733QS-HG003T),319091,https://www.e-katalog.ru/ASUS-G733QS-HG003T.htm,17.3,1920,300,250,8,16.0,3.3,4.6,4096,16,23412,32,64,1,3200,2,1,8.0,2048,0,3,0,0,1,0,4.0,90.0,12.00,2.70,2,14317,0,0,0,2.0,395,...,0.223607,-0.5,-0.670820,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.597614,0.545545,-0.372678,0.188982,-0.062994,-4.433780e-17,-0.816497,-0.298807,0.098606,0.175483,-0.341733,0.300960,-0.084231,-0.185507,0.379950,-0.436039,0.371110,-0.248910,0.132074,-0.053325,0.014367,-0.077152,-0.385758,0.184637,0.362620,-0.320970,-0.307729,0.597440
3459,/LENOVO-5-17ITH6-82JN0009RU.htm,Lenovo Legion 5 17ITH6 (5 17ITH6 82JN0009RU),105000,https://www.e-katalog.ru/LENOVO-5-17ITH6-82JN0...,17.3,1920,144,300,6,12.0,2.7,4.5,7680,12,15926,16,16,1,3200,2,1,4.0,512,0,4,0,0,0,1280,2.0,80.0,7.50,2.98,3,12041,0,2,2,3.0,399,...,0.223607,-0.5,-0.670820,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.597614,0.545545,-0.372678,0.188982,-0.062994,-4.433780e-17,-0.816497,-0.298807,0.098606,0.175483,-0.341733,0.300960,-0.084231,-0.185507,0.379950,-0.436039,0.371110,-0.248910,0.132074,-0.053325,0.014367,-0.385758,0.077152,0.307729,-0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3388 entries, 0 to 3709
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3388 non-null   object 
 1   name            3388 non-null   object 
 2   price           3388 non-null   int64  
 3   link            3388 non-null   object 
 4   diagonal        3388 non-null   float64
 5   display_res     3388 non-null   int64  
 6   frame_rate      3388 non-null   int64  
 7   bright          3388 non-null   int64  
 8   cores           3388 non-null   int64  
 9   threads         3388 non-null   float64
 10  clock_freq      3388 non-null   float64
 11  turbo_freq      3388 non-null   float64
 12  l2_cashe        3388 non-null   int64  
 13  l3_cashe        3388 non-null   int64  
 14  passmark_test   3388 non-null   int64  
 15  ram_vol         3388 non-null   int64  
 16  max_ram_vol     3388 non-null   int64  
 17  ddr4            3388 non-null   i

In [18]:
note.iloc[:,38].value_counts()

0.0    1418
1.0     792
2.0     708
3.0     449
4.0      21
Name: max_monitors, dtype: int64